In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.controller.robot_config import RobotType
from pkg.utils.test_scripts import *
from pkg.planning.filtering.filter_interface import *
from pkg.planning.constraint.constraint_common import BindingTransform

ROBOT_TYPE = RobotType.indy7
VISUALIZE = True

ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## check GraspChecker scene

In [2]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)
gcheck.show_vertices = True

cpath = os.path.join(SCENE_PATH, "GraspChecker")
files = sorted(os.listdir(cpath))
print(files)

['00000000-OK.pkl', '00000001-OK.pkl', '00000002-OK.pkl', '00000003-OK.pkl', '00000004-OK.pkl', '00000005-OK.pkl', '00000006-OK.pkl', '00000007-OK.pkl', '00000008-OK.pkl', '00000009-OK.pkl', '00000010-OK.pkl', '00000011-OK.pkl', '00000012-OK.pkl', '00000013-OK.pkl']


In [9]:
file_path = os.path.join(cpath, files[7])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)

res = gcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result))

file_path: /home/rnb/Projects/rnb-planning/data/checker_scenes/GraspChecker/00000007-OK.pkl
res: True (0.03)
wait for key
result/log: True / True


## check ReachChecker scene

In [10]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

cpath = os.path.join(SCENE_PATH, "ReachChecker")
files = sorted(os.listdir(cpath))
print(files)

['00000000-OK.pkl', '00000001-OK.pkl', '00000002-OK.pkl', '00000003-OK.pkl', '00000004-OK.pkl', '00000005-OK.pkl']


In [20]:
file_path = os.path.join(cpath, files[0])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)

res = rcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result))

file_path: /home/rnb/Projects/rnb-planning/data/checker_scenes/ReachChecker/00000000-OK.pkl
result/log: True / True


## check LatticedChecker scene

In [14]:
from pkg.planning.filtering.latticized_filter import LatticedChecker
lcheck = LatticedChecker(pscene, gcheck=gcheck)

cpath = os.path.join(SCENE_PATH, "LatticedChecker")
files = sorted(os.listdir(cpath))
print(files)

from pkg.planning.filtering.latticized_filter import ARM_SHAPE, GRASP_SHAPE, \
    WDH_GRASP, L_CELL_GRASP, OFFSET_ZERO_GRASP, WDH_ARM, L_CELL_ARM, OFFSET_ZERO_ARM
from pkg.planning.filtering.lattice_model.latticizer_py import get_centers

centers_grasp = get_centers(GRASP_SHAPE, L_CELL_GRASP, OFFSET_ZERO_GRASP)
centers_arm = get_centers(ARM_SHAPE, L_CELL_ARM, OFFSET_ZERO_ARM)

assert len(pscene.robot_chain_dict.values())==1, "not single robot scene"
shoulder_height = lcheck.shoulder_height_dict[ROBOT_NAME]

===== WARNING: latticized_filter in DEBUG MODE====
['00000000-OK.pkl', '00000001-OK.pkl', '00000002-OK.pkl', '00000003-OK.pkl', '00000004-OK.pkl']


In [37]:
file_path = os.path.join(cpath, files[4])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result[1]>0.5))

file_path: /home/rnb/Projects/rnb-planning/data/checker_scenes/LatticedChecker/00000004-OK.pkl
result/log: True / True


In [38]:
grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
grasp_tool_img = grasp_img[:,:,:,0]
grasp_obj_img = grasp_img[:,:,:,1]
grasp_tar_img = grasp_img[:,:,:,2]
arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

grasp_tool_idx = np.where(grasp_tool_img)
grasp_obj_idx = np.where(grasp_obj_img)
grasp_tar_idx = np.where(grasp_tar_img)
arm_tar_idx = np.where(arm_img)
radius, theta, height = lcheck.rth_last
offset_ee = cyl2cart(radius, theta, height)
print("r, h match: {}".format(np.equal(rh_vals, [radius, height])))

for i_c, center in enumerate(centers_arm[arm_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
    gscene.create_safe(GEOTYPE.SPHERE, "arm_%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0.5, 0.5, 0.5, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_tool_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tool_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)

for i_c, center in enumerate(centers_grasp[grasp_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tar_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 0, 0, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_obj_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "obj_%05d"%(i_c), "base_link", dims=(0.025,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)

r, h match: [ True  True]


### find idx that deals with object other than obj_0

In [4]:
for file_idx in range(1173):
    scene_data = load_pickle(os.path.join(SCENE_PATH, "{:08d}-OK.pkl".format(file_idx)))
    global_log = scene_data["global_log"]
#     if global_log["subject"] != "obj_0" and global_log["actor"] != "grip0":
    if global_log["actor"] != "grip0":
        print(file_idx)

1
3
5
7
9
11
13
15
17
19
21
24
26
27
28
30
32
34
36
38
40
42
44
46
48
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
94
96
98
100
102
104
106
108
110
112
114
116
119
121
123
125
127
131
133
135
137
139
141
143
145
147
149
151
153
156
158
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
192
194
196
198
207
208
210
212
214
216
218
220
222
225
271
290
296
306
343
349
361
381
415
416
417
418
419
420
421
486
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
561
566
586
589
591
596
612


KeyboardInterrupt: 